## From Database back into df (querying, etc.)


https://docs.sqlalchemy.org/en/14/tutorial/data.html

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
engine = create_engine("sqlite:///vaers.db")
engine

Engine(sqlite:///vaers.db)

In [3]:
all_tables = engine.table_names()
all_tables

['data', 'symptoms', 'vax']

### SQL statements
VAERS data guide: https://vaers.hhs.gov/docs/VAERSDataUseGuide_en_September2021.pdf

Limiting to data for Covid-19 vaccines
- 1) Dataframe with: vaccine manufacturer, dose series number, vaccination date, number of days (calculated interval (in days) from the vaccination date to the onset date), age of patient (in years), and the Vaers ID for report
- 2) Dataframe with: symptoms, vaccine manufacturer, dose series number, year (of report)

Will lightly clean (remove nulls, group categories) before saving back to SQLite

In [48]:
#first query

s1 = '''
SELECT v.vax_manu, v.vax_dose_series, d.v_adminby, d.vax_date, d.numdays, d.age_yrs, v.vaers_id
FROM vax v 
INNER JOIN data d 
ON v.vaers_id = d.vaers_id
WHERE v.vax_type = 'COVID19';
'''

In [49]:
df1 = pd.read_sql(s1, engine)

In [50]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811629 entries, 0 to 811628
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   vax_manu         811629 non-null  object 
 1   vax_dose_series  808680 non-null  object 
 2   v_adminby        811629 non-null  object 
 3   vax_date         758782 non-null  object 
 4   numdays          718031 non-null  float64
 5   age_yrs          735131 non-null  float64
 6   vaers_id         811629 non-null  int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 43.3+ MB


In [51]:
df1.head()

,vax_manu,vax_dose_series,v_adminby,vax_date,numdays,age_yrs,vaers_id
0,PFIZER\BIONTECH,1,PVT,12/15/2020,0.0,56.0,902418
1,PFIZER\BIONTECH,1,PVT,12/15/2020,0.0,35.0,902440
2,PFIZER\BIONTECH,1,OTH,12/15/2020,0.0,55.0,902446
3,PFIZER\BIONTECH,UNK,PVT,12/15/2020,0.0,42.0,902464
4,PFIZER\BIONTECH,1,PUB,12/15/2020,0.0,60.0,902465


In [51]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811629 entries, 0 to 811628
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   vax_manu         811629 non-null  object 
 1   vax_dose_series  808680 non-null  object 
 2   v_adminby        811629 non-null  object 
 3   numdays          718031 non-null  float64
 4   age_yrs          735131 non-null  float64
 5   vaers_id         811629 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 37.2+ MB


In [52]:
#dropping nulls
df1 = df1.dropna(subset = ['vax_dose_series', 'v_adminby', 'numdays', 'age_yrs', 'vax_date'])

In [53]:
df1.v_adminby.value_counts()

PVT    175303
PHM    152990
UNK    150065
OTH     81239
PUB     76136
WRK     25477
SEN     10960
SCH     10770
MIL      9067
Name: v_adminby, dtype: int64

In [54]:
dict_location = {'PVT': 'Private', 'PHM': 'Pharmacy', 'MIL': 'Military', 'OTH': 'Other', 'PUB': 'Public', 'WRK': 'Workplace Clinic', 'UNK': 'Unknown', 'OTH': 'Other', 'SEN': 'Nursing Home', 'SCH': 'School'}

In [55]:
df1.replace({'v_adminby': dict_location}, inplace=True)

In [56]:
df1.vax_dose_series.value_counts()

1      335737
2      210814
UNK     92444
3       51618
4         782
7+        309
5         194
6         109
Name: vax_dose_series, dtype: int64

In [57]:
def group_multdoses(dose):
    if dose in ['5','6','7+']:
        dose = '5+'
    return dose

In [58]:
df1.vax_dose_series = df1.vax_dose_series.apply(lambda x: group_multdoses(x))

In [59]:
df1.vax_dose_series.value_counts()

1      335737
2      210814
UNK     92444
3       51618
4         782
5+        612
Name: vax_dose_series, dtype: int64

In [61]:
# df1.to_pickle('df1.pkl')

In [35]:
# query for side effects table for vaers_id that matches for covid-19 vax

s2 = '''

SELECT vaers_id, year,
GROUP_CONCAT(COALESCE(symptom1, ''),', ') ||', '||
GROUP_CONCAT(COALESCE(symptom2, ''),', ') ||', '||
GROUP_CONCAT(COALESCE(symptom3, ''),', ') ||', '||
GROUP_CONCAT(COALESCE(symptom4, ''),', ') ||', '||
GROUP_CONCAT(COALESCE(symptom5, ''), ', ') AS all_symptoms
FROM symptoms 
WHERE vaers_id IN
(SELECT vaers_id 
FROM vax WHERE vax_type = 'COVID19')
GROUP BY vaers_id;
'''

In [36]:
symptoms_df = pd.read_sql(s2, engine)

In [37]:
symptoms_df

,vaers_id,year,all_symptoms
0,902418,2020,"Hypoaesthesia, Injection site hypoaesthesia, , ,"
1,902440,2020,"Headache, , , ,"
2,902446,2020,"Erythema, Feeling hot, Flushing, ,"
3,902464,2020,"Dizziness, Electrocardiogram normal, Hyperhidr..."
4,902465,2020,"Dysgeusia, Sensory disturbance, Oral pruritus,..."
...,...,...,...
774365,2139618,2022,"COVID-19, SARS-CoV-2 test positive, , ,"
774366,2139619,2022,"COVID-19, SARS-CoV-2 test positive, Vaccine br..."
774367,2139620,2022,"Hypoaesthesia, , , ,"
774368,2139621,2022,"Alanine aminotransferase normal, Antinuclear a..."


In [38]:
symptoms_df.vaers_id.nunique()

774370

In [63]:
symptoms_df.iloc[2].all_symptoms

'Erythema, Feeling hot, Flushing, , '

In [60]:
# getting more detailed df with symptoms, year, vaccine, and dose number

s3 = '''

SELECT s.vaers_id, s.year, v.vax_manu, v.vax_dose_series,
GROUP_CONCAT(COALESCE(s.symptom1, ''),', ') ||', '||
GROUP_CONCAT(COALESCE(s.symptom2, ''),', ') ||', '||
GROUP_CONCAT(COALESCE(s.symptom3, ''),', ') ||', '||
GROUP_CONCAT(COALESCE(s.symptom4, ''),', ') ||', '||
GROUP_CONCAT(COALESCE(s.symptom5, ''), ', ') AS all_symptoms
FROM symptoms s
INNER JOIN vax v ON
s.vaers_id = v.vaers_id
WHERE v.vax_type = 'COVID19'
GROUP BY s.vaers_id;
'''

In [61]:
df3 = pd.read_sql(s3, engine)

In [62]:
df3

,vaers_id,year,vax_manu,vax_dose_series,all_symptoms
0,902418,2020,PFIZER\BIONTECH,1,"Hypoaesthesia, Injection site hypoaesthesia, , ,"
1,902440,2020,PFIZER\BIONTECH,1,"Headache, , , ,"
2,902446,2020,PFIZER\BIONTECH,1,"Erythema, Feeling hot, Flushing, ,"
3,902464,2020,PFIZER\BIONTECH,UNK,"Dizziness, Electrocardiogram normal, Hyperhidr..."
4,902465,2020,PFIZER\BIONTECH,1,"Dysgeusia, Sensory disturbance, Oral pruritus,..."
...,...,...,...,...,...
774365,2139618,2022,UNKNOWN MANUFACTURER,UNK,"COVID-19, SARS-CoV-2 test positive, , ,"
774366,2139619,2022,MODERNA,3,"COVID-19, SARS-CoV-2 test positive, Vaccine br..."
774367,2139620,2022,JANSSEN,2,"Hypoaesthesia, , , ,"
774368,2139621,2022,MODERNA,2,"Alanine aminotransferase normal, Antinuclear a..."


In [64]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774370 entries, 0 to 774369
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   vaers_id         774370 non-null  int64 
 1   year             774370 non-null  int64 
 2   vax_manu         774370 non-null  object
 3   vax_dose_series  771476 non-null  object
 4   all_symptoms     774370 non-null  object
dtypes: int64(2), object(3)
memory usage: 29.5+ MB


In [65]:
#dropping nulls
df3 = df3.dropna(subset = ['vax_manu', 'vax_dose_series'])

In [68]:
df3.vax_dose_series = df3.vax_dose_series.apply(lambda x: group_multdoses(x))

/Users/zliao/opt/anaconda3/envs/metis/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [70]:
df3.vax_dose_series.value_counts()

1      386707
2      207190
UNK    121865
3       54324
4         771
5+        619
Name: vax_dose_series, dtype: int64

### Saving dataframes back to SQLite as tables 
Saving these semi-cleaned dataframes back to SQL as new tables. The rest of the cleaning will be done via Spark.

In [71]:
df1.to_sql('stats', con=engine, if_exists = 'replace', index=False)

In [72]:
df3.to_sql('ade', con=engine,if_exists = 'replace', index=False)

Saving to CSV files too, for analysis with Spark:

In [73]:
df1.to_csv('stats.csv')
df3.to_csv('ade.csv')

## First analyzing with Pandas

Sources:
https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
https://proinsias.github.io/til/Pandas-Named-Aggregation/

In [65]:
import numpy as np

In [71]:
# summary stats

summary_df1 = df1.groupby(['vax_manu', 'vax_dose_series', 'v_adminby']).agg(
    avg_onset_days = pd.NamedAgg(column='numdays', aggfunc='mean'),
    std_onset_days= pd.NamedAgg(column='numdays', aggfunc='std'),
    avg_age = pd.NamedAgg(column='age_yrs', aggfunc='mean'),
    std_age = pd.NamedAgg(column='age_yrs', aggfunc='std'),
    cases = pd.NamedAgg(column='vaers_id', aggfunc='count')).reset_index()


In [75]:
summary_df1.to_pickle('summary_df1.pkl')

In [77]:
summary_df1.vax_manu.unique()

array(['JANSSEN', 'MODERNA', 'PFIZER\\BIONTECH', 'UNKNOWN MANUFACTURER'],
      dtype=object)